In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.gridspec import GridSpec
from matplotlib.patches import Rectangle
from scipy.stats import gaussian_kde
import numpy as np

def get_pitcher_name():
    first_name = input("Enter the pitcher's first name: ")
    last_name = input("Enter the pitcher's last name: ")
    return last_name + ", " + first_name

def get_pitch_heatmap(df, Pitcher, pitch_type):
    if pitch_type == "Breaking Ball":
        pitch_df = df.loc[(df['Pitcher'] == Pitcher) & (df['TaggedPitchType'].isin(["Slider", "Curveball"]))]
    elif pitch_type == "Fastball":
        pitch_df = df.loc[(df['Pitcher'] == Pitcher) & (df['TaggedPitchType'].isin(["Fastball","Four-Seam", "Sinker", "Cutter"]))]
    elif pitch_type == "Offspeed":
        pitch_df = df.loc[(df['Pitcher'] == Pitcher) & (df['TaggedPitchType'].isin(["ChangeUp", "Splitter"]))]

    x = -pitch_df['PlateLocSide']
    y = pitch_df['PlateLocHeight']
    k = gaussian_kde(np.vstack([x, y]))
    xi, yi = np.mgrid[-2:2:100j, 0:5:100j]
    zi = k(np.vstack([xi.flatten(), yi.flatten()]))

    return xi, yi, zi.reshape(xi.shape)

def generate_report(df, pitcher_name):
    pitcher_data = df[df['Pitcher'] == pitcher_name]
    first_name, last_name = pitcher_name.split(', ')
    formatted_pitcher_name = f"{last_name} {first_name}"

    pitch_type_counts = pitcher_data['TaggedPitchType'].value_counts()
    pitch_type_averages = pitcher_data.groupby('TaggedPitchType')[['RelSpeed', 'SpinRate', 'InducedVertBreak', 'HorzBreak', 'RelHeight', 'RelSide', 'Extension']].mean()

    strikes = pitcher_data[pitcher_data['PitchCall'].isin(['StrikeCalled', 'StrikeSwinging', 'FoulBall','InPlay'])]
    strike_percentages = strikes.groupby('TaggedPitchType')['PitchCall'].count() / pitch_type_counts * 100

    hits = len(pitcher_data[pitcher_data['PlayResult'].isin(['Single', 'Double', 'Triple', 'HomeRun'])])
    total_pitches = len(pitcher_data)
    strikeouts = len(pitcher_data[pitcher_data['KorBB'] == 'Strikeout'])
    walks = len(pitcher_data[pitcher_data['KorBB'] == 'Walk'])

    outs_pitch = (pitcher_data['Outs'].diff() > 0).sum()
    outs_play = len(pitcher_data[pitcher_data['PlayResult'].isin(['Out', 'Sacrifice'])])
    total_outs = outs_pitch + outs_play

    innings_pitched = "5"

    first_pitches = pitcher_data[pitcher_data['PitchofPA'] == 1]
    first_pitch_strikes = first_pitches[first_pitches['PitchCall'].isin(['StrikeCalled', 'StrikeSwinging', 'FoulBall','InPlay'])]
    first_pitch_strike_percentage = len(first_pitch_strikes) / len(first_pitches) * 100
    
    df = df.replace([np.inf, -np.inf], np.nan)
    #if NaN, take mean
    df = df.fillna(df.mean())
    

    fastball_heatmap = get_pitch_heatmap(df, pitcher_name, 'Fastball')
    breakingball_heatmap = get_pitch_heatmap(df, pitcher_name, 'Breaking Ball')
    df['PlateLocSide'].replace([np.inf, -np.inf], np.nan, inplace=True)
    df['PlateLocSide'].fillna(df['PlateLocSide'].median(), inplace=True)

    df['PlateLocHeight'].replace([np.inf, -np.inf], np.nan, inplace=True)
    df['PlateLocHeight'].fillna(df['PlateLocHeight'].median(), inplace=True)
    offspeed_heatmap = get_pitch_heatmap(df, pitcher_name, 'Offspeed')
    
    swinging_strikes = pitcher_data[pitcher_data['PitchCall'] == 'StrikeSwinging'].groupby('TaggedPitchType').size()
    total_strikes = pitcher_data[pitcher_data['PitchCall'].isin(['InPlay', 'StrikeCalled','StrikeSwinging', 'FoulBall'])].groupby('TaggedPitchType').size()
    whiff_percentages = swinging_strikes / total_strikes * 100
    #if whiff is nan, set to 0
    whiff_percentages = whiff_percentages.fillna(0)


    with PdfPages("/Users/johndavis/Desktop/pitcher_report.pdf") as pdf:
        fig = plt.figure(figsize=(11, 17))
        gs = GridSpec(nrows=4, ncols=2, height_ratios=[1, 2, 2, 2], wspace=0.2, hspace=0.3)
        fig.patch.set_facecolor('xkcd:light grey')

        ax1 = fig.add_subplot(gs[0, :])
        ax1.pie(pitch_type_counts, labels=pitch_type_counts.index, autopct='%1.1f%%', startangle=90)
        ax1.set_title('Pitch Types Distribution', fontsize=16)
        ax1.set_position([0.175, 0.76, 0.35, 0.15])

        ax2 = fig.add_subplot(gs[1, 0])
        ax2.axis('off')
        basic_stats_data = [
            ['Total Pitches', total_pitches],
            ['Innings Pitched', innings_pitched],
            ['Strikeouts', strikeouts],
            ['Walks', walks],
            ['Hits', hits],
            ['First Pitch Strike %', round(first_pitch_strike_percentage, 2)]
        ]
        basic_stats_table = ax2.table(cellText=basic_stats_data, cellLoc='left', loc='center', colLabels=None)
        basic_stats_table.auto_set_font_size(False)
        basic_stats_table.set_fontsize(14)
        basic_stats_table.auto_set_column_width(col=list(range(2)))
        for (i, j), cell in basic_stats_table._cells.items():
            if i == 0:
                cell.set_text_props(weight='bold', color='white')
                cell.set_facecolor('gray')
            cell.set_height(0.2)
        ax2.set_position([0.52, 0.79, 0.35, 0.12])

        ax3 = fig.add_subplot(gs[1, 1:])
        ax3.axis('off')
        pitch_type_averages_table = ax3.table(cellText=pitch_type_averages.round(1).values,
                                            colLabels=pitch_type_averages.columns,
                                            rowLabels=pitch_type_averages.index,
                                            cellLoc='center', loc='center')
        pitch_type_averages_table.auto_set_font_size(False)
        pitch_type_averages_table.set_fontsize(14)
        pitch_type_averages_table.auto_set_column_width(col=list(range(len(pitch_type_averages.columns))))
        for (i, j), cell in pitch_type_averages_table._cells.items():
            if i == 0:
                cell.set_text_props(weight='bold', color='white')
                cell.set_facecolor('gray')
            cell.set_height(0.2)
        #ax3.set_title(f"{formatted_pitcher_name} - Pitch Level Metrics", fontsize=16)
        ax3.set_position([0.52, 0.27, 0.35, 0.12])

        ax4 = fig.add_subplot(gs[2, :])
        strike_zone = Rectangle((-0.83, 1.5), 1.66, 2.1, linewidth=1, edgecolor='k', facecolor='none')
        ax4.add_patch(strike_zone)
        ax4.set_xlim(-2, 2)
        ax4.set_ylim(0, 4.5)
        ax4.set_aspect('equal')
        colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']

        for idx, pitch_type in enumerate(pitch_type_counts.index):
            filtered_data = pitcher_data[pitcher_data['TaggedPitchType'] == pitch_type]
            ax4.scatter(filtered_data['PlateLocSide'], filtered_data['PlateLocHeight'], s=80, c=colors[idx], alpha=0.5, label=pitch_type)
        ax4.legend(title="Pitch Types", fontsize=16)
        ax4.set_title('Pitch Locations', fontsize=20)
        ax4.set_position([0.12, 0.43, 0.55, 0.32])

        ax5 = fig.add_subplot(gs[2, 1:])
        ax5.axis('off')
        strike_percentages_data = [[pitch_type, round(strike_percentages[pitch_type], 2)] for pitch_type in strike_percentages.index]
        strike_percentages_table = ax5.table(cellText=strike_percentages_data,
                                            colLabels=['Pitch Type', 'Strike %'],
                                            cellLoc='center', loc='center')
        strike_percentages_table.auto_set_font_size(False)
        strike_percentages_table.set_fontsize(14)
        strike_percentages_table.auto_set_column_width(col=list(range(2)))
        for (i, j), cell in strike_percentages_table._cells.items():
            if i == 0:
                cell.set_text_props(weight='bold', color='white')
                cell.set_facecolor('gray')
            cell.set_height(0.2)
        ax5.set_position([0.825, 0.775, 0.35, 0.15])

        ax6 = fig.add_subplot(gs[2, 0])
        ax6.axis('off')
        ax6.set_title('Fastball Heatmap', fontsize=16)
        ax6.imshow(fastball_heatmap[2], cmap='Oranges', interpolation='nearest', extent=[-2, 2, 0, 5], origin='lower')
        ax6.set_aspect('equal')
        ax6.set_position([0.575, 0.6, 0.35, 0.15])
        #add a rectangle to the plot of the heatmap to show the strike zone
        ax6.add_patch(Rectangle((-0.83, 1.5), 1.66, 2.1, linewidth=1, edgecolor='k', facecolor='none'))
        #flip the x-axis so that the plot is oriented the same way as the strike zone
        ax6.invert_yaxis()
        #shidt the heatmap 90 degrees so the it is turned 90 degrees clockwise
        ax6.imshow(np.rot90(fastball_heatmap[2],3), cmap='Oranges', interpolation='nearest', extent=[-2, 2, 0, 5], origin='lower')
        
        
        
        

        ax7 = fig.add_subplot(gs[2, 1])
        ax7.axis('off')
        ax7.set_title('Breaking Ball Heatmap', fontsize=16)
        ax7.imshow(breakingball_heatmap[2], cmap='Oranges', interpolation='nearest', extent=[-2, 2, 0, 5], origin='lower')
        ax7.set_aspect('equal')
        ax7.set_position([0.855, 0.6, 0.35, 0.15])
        #add a rectangle to the plot of the heatmap to show the strike zone
        ax7.add_patch(Rectangle((-0.83, 1.5), 1.66, 2.1, linewidth=1, edgecolor='k', facecolor='none'))
        #flip the x-axis so that the plot is oriented the same way as the strike zone
        ax7.invert_yaxis()
        #shidt the heatmap 90 degrees so the it is turned 90 degrees clockwise
        ax7.imshow(np.rot90(breakingball_heatmap[2],3), cmap='Oranges', interpolation='nearest', extent=[-2, 2, 0, 5], origin='lower')
       
        
    

        ax8 = fig.add_subplot(gs[3, 0])
        ax8.axis('off')
        ax8.set_title('Offspeed Heatmap', fontsize=16)
        ax8.imshow(offspeed_heatmap[2], cmap='Oranges', interpolation='nearest', extent=[-2, 2, 0, 5], origin='lower')
        ax8.set_aspect('equal')
        ax8.set_position([0.85, 0.42, 0.35, 0.15])
        #add a rectangle to the plot of the heatmap to show the strike zone
        ax8.add_patch(Rectangle((-0.83, 1.5), 1.66, 2.1, linewidth=1, edgecolor='k', facecolor='none'))
        #flip the x-axis so that the plot is oriented the same way as the strike zone
        ax8.invert_yaxis()
        #shidt the heatmap 270 degrees so the it is turned 90 degrees clockwise
        ax8.imshow(np.rot90(offspeed_heatmap[2], 3), cmap='Oranges', interpolation='nearest', extent=[-2, 2, 0, 5], origin='lower')
        #ax8.imshow(np.rot90(offspeed_heatmap[2], 1), cmap='Oranges', interpolation='nearest', extent=[-2, 2, 0, 5], origin='lower')
        
        ax9 = fig.add_subplot(gs[3, 1:])  # Modify as needed for your layout.
        ax9.axis('off')
        whiff_percentages_data = [[pitch_type, round(whiff_percentages[pitch_type], 2)] for pitch_type in whiff_percentages.index]
        whiff_percentages_table = ax9.table(cellText=whiff_percentages_data,
                                            colLabels=['Pitch Type', 'Whiff %'],
                                            cellLoc='center', loc='center')
        whiff_percentages_table.auto_set_font_size(False)
        whiff_percentages_table.set_fontsize(14)
        whiff_percentages_table.auto_set_column_width(col=list(range(2)))
        for (i, j), cell in whiff_percentages_table._cells.items():
            if i == 0:
                cell.set_text_props(weight='bold', color='white')
                cell.set_facecolor('gray')
            cell.set_height(0.2)
        ax9.set_position([0.59, 0.425, 0.35, 0.15])
        
        

        plt.suptitle(f"{formatted_pitcher_name} - Post Game Report", fontsize=32)
        pdf.savefig(fig, bbox_inches='tight')
        pdf.close()

df = pd.read_csv("/Users/johndavis/Desktop/June_20th.csv", low_memory=False)

pitcher_name = get_pitcher_name()
generate_report(df, pitcher_name)